# ensemble_methods

In [8]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

seed = 42

In [3]:
import nbimporter

import pre_processing
import feature_selection
import feature_generation
import parameter_tuning

In [26]:
def escribir_respuesta(ids,predicciones):
    with open("respuesta.csv",'w') as archivo:
        archivo.write("id,target\n")
        for i in range(len(ids)):
            linea = f"{int(ids[i])},{predicciones[i]}"
            archivo.write(f"{linea}\n")

# Bagging para LightGBM

In [4]:
train,test = pre_processing.load_featured_datasets()

In [5]:
params = parameter_tuning.get_best_params()

In [14]:
from sklearn.ensemble import BaggingRegressor
import lightgbm as lgb

In [9]:
train['precio'] = train['precio'].map(lambda x: math.log(x))

In [10]:
train_selected = feature_selection.get_selected_dataframe(train)

In [11]:
X = train_selected.drop('precio', axis=1)
Y = train_selected['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [21]:
params['lightgbm']

{'num_leaves': 55,
 'min_gain_to_split': 0.2,
 'min_data_in_leaf': 3000,
 'max_depth': 12,
 'max_bin': 150,
 'feature_fraction': 0.7,
 'bagging_freq': 5,
 'bagging_fraction': 0.75,
 'boosting_type': 'gbdt',
 'objective': 'regression',
 'metric': 'mae',
 'num_boost_round': 5000,
 'verbose': 0,
 'learning_rate': 0.1}

In [22]:
reg = lgb.LGBMRegressor(num_leaves= 55, min_gain_to_split= 0.07, min_data_in_leaf= 3000,
                        max_depth= 12, max_bin= 150, feature_fraction= 0.7, bagging_freq= 5,
                        bagging_fraction= 0.75, boosting_type='gbdt', objective= 'regression',
                        metric = 'mae', num_boost_round= 5000, verbose= 0, learning_rate= 0.1)

In [23]:
bagging = BaggingRegressor(reg, random_state=seed)

In [24]:
bagging.fit(X,Y)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:148: UserWarning: Foun

BaggingRegressor(base_estimator=LGBMRegressor(bagging_fraction=0.75,
                                              bagging_freq=5,
                                              boosting_type='gbdt',
                                              class_weight=None,
                                              colsample_bytree=1.0,
                                              feature_fraction=0.7,
                                              importance_type='split',
                                              learning_rate=0.1, max_bin=150,
                                              max_depth=12, metric='mae',
                                              min_child_samples=20,
                                              min_child_weight=0.001,
                                              min_data_in_leaf=3000,
                                              min_gain_to_split=0.07,
                                              min_split_gain=0.0,
                                  

In [25]:
Y_pred = bagging.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

427622.7929542567

In [27]:
test_selected = feature_selection.get_selected_dataframe(test, precio=False)

In [28]:
ids = test_selected.index.values
X_test = test_selected.values
test_predict = bagging.predict(X_test)
f = np.vectorize(math.exp)
test_predict = f(test_predict)
escribir_respuesta(ids, test_predict)